# Julia言語で入門するプログラミング

[Julia言語で入門するプログラミング](https://muuumin.net/julia-intro-prog/)

## 最初に作るゲーム

- Simple RPG
    - 勇者
        - 攻撃力: 10
    - モンスター
        - HP: 30
    - 行動(勇者)
        - モンスターを攻撃
            - モンスター.HP を 勇者.攻撃力 の分だけ減らす

In [2]:
function main()
    モンスターHP = 30
    勇者攻撃力 = 10

    println("モンスターに遭遇した！")
    println("戦闘開始！")

    for _ in 1:3
        println("----------")
        println("勇者の攻撃！")
        モンスターダメージ = 勇者攻撃力
        モンスターHP = モンスターHP - モンスターダメージ
        println("モンスターは $(モンスターダメージ) のダメージを受けた！")
        println("モンスターの残りHP: $(モンスターHP)")
    end
end

main()

モンスターに遭遇した！
戦闘開始！
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP: 20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP: 10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP: 0


## 自動テスト

上記コードは、ダメージ計算を直書きしているが、こういったコードはダメージ計算の仕方を変更する場合などにバグを起こしやすい

変更に強いコードを書くためには自動テストが必要である

- **自動テスト**:
    - 入力に対して期待する出力を事前に定めておき、書いたコードが期待通りであることを保証するテスト
    - コードの変更が入る度に実行し、プログラムを破壊していないことを確認する

### コードの関数化
自動テストは通常、ある入力に対する出力をテストする

そのため、テストしたいコードは関数化する必要がある

また、その関数は同じ入力に対する出力が変わらない、つまり **参照透過性** を持つ必要がある

関数を実行する度に出力が変わってしまっては、事前に入力と出力の期待値を定めることができないためである

In [3]:
モンスターHP = 30
勇者攻撃力 = 10

# ❌ 参照透過性のない関数
## 同じ引数 10 を渡しても、実行する度に出力値が変わってしまう
function モンスターダメージ(ダメージ量)
    global モンスターHP -= ダメージ量
end

モンスターダメージ(勇者攻撃力) |> println # => 20
モンスターダメージ(勇者攻撃力) |> println # => 10
モンスターダメージ(勇者攻撃力) |> println # => 0

20
10
0


In [4]:
モンスターHP = 30
勇者攻撃力 = 10

# ⭕ 参照透過性のある関数
## 同じ引数を渡せば、常に同じ値が出力される
function モンスターダメージ(モンスターHP, ダメージ量)
    ダメージ後のモンスターHP = モンスターHP - ダメージ量
end

モンスターダメージ(モンスターHP, 勇者攻撃力) |> println # => 20
モンスターダメージ(モンスターHP, 勇者攻撃力) |> println # => 20
モンスターダメージ(モンスターHP, 勇者攻撃力) |> println # => 20

20
20
20


In [5]:
# テストパッケージ読み込み
using Test

"""
    ダメージ計算(攻撃力::Int) -> ダメージ量::Int

ダメージ計算関数
"""
function ダメージ計算(攻撃力::Int)
    ダメージ量 = 攻撃力
end

"""
    HP計算(HP::Int, ダメージ量::Int) -> ダメージ後のHP::Int

HP計算関数
"""
function HP計算(HP::Int, ダメージ量::Int)
    ダメージ後のHP = HP - ダメージ量
end

#=
テスト実行

- Testケース
    - ダメージ計算
        - 攻撃力が 10 のとき、ダメージ量は 10 になって欲しい
        - 攻撃力が -10 のとき、ダメージ量は -10 になって欲しい
    - HP計算
        - HPが 20, ダメージ量が 10 のとき、ダメージ後のHPは 10 になって欲しい
        - HPが 10, ダメージ量が 10 のとき、ダメージ後のHPは 0 になって欲しい
        - HPが 0, ダメージ量が 10 のとき、ダメージ後のHPは -10 になって欲しい
- Test関連マクロ
    - @testset begin テストコード end
    - @test 期待値 => 期待値通りであればテストが通る
=#
@testset "ダメージ計算" begin
    @test ダメージ計算(10) === 10   # 10 が入力されれば 10 が出力されるべき
    @test ダメージ計算(-10) === -10 # -10 が入力されれば -10 が出力されるべき
end

@testset "HP計算" begin
    @test HP計算(20, 10) === 10  # HP: 20, ダメージ量: 10 が入力されれば 10 が出力されるべき
    @test HP計算(10, -10) === 20 # HP: 10, ダメージ量: -10 が入力されれば 20 が出力されるべき
    @test HP計算(0, 10) === -10  # HP: 0, ダメージ量: 10 が入力されれば -10 が出力されるべき
end

Test Summary: | Pass  Total
ダメージ計算  |    2      2
Test Summary: | Pass  Total
HP計算        |    3      3


Test.DefaultTestSet("HP計算", Any[], 3, false, false)

### ダメージ計算関数の修正
ここで仕様変更が入り、ダメージ計算を行う際に、防御力を考慮するように修正することになった

ダメージ計算関数を修正し、テストにより関数の保証を行った上で、メインプログラムを実行する

In [8]:
# テストパッケージ読み込み
using Test

"""
    ダメージ計算(攻撃力::Int, 防御力::Int) -> ダメージ量::Int

ダメージ計算関数（防御力考慮バージョン）
"""
function ダメージ計算(攻撃力::Int, 防御力::Int)
    ダメージ量 = round(Int, 10 * 攻撃力/ 防御力)
end

"""
    HP計算(HP::Int, ダメージ量::Int) -> ダメージ後のHP::Int

HP計算関数
"""
function HP計算(HP::Int, ダメージ量::Int)
    ダメージ後のHP = HP - ダメージ量
end

# テスト実行
@testset "ダメージ計算" begin
    @test ダメージ計算(10, 5) === 20   # 攻撃力: 10, 防御力: 5 が入力されれば 20 が出力されるべき
    @test ダメージ計算(10, 10) === 10  # 攻撃力: 10, 防御力: 10 が入力されれば 10 が出力されるべき
    @test ダメージ計算(-10, -20) === 5 # 攻撃力: -10, 防御力: -20 が入力されれば 5 が出力されるべき
end

@testset "HP計算" begin
    @test HP計算(20, 10) === 10  # HP: 20, ダメージ量: 10 が入力されれば 10 が出力されるべき
    @test HP計算(10, -10) === 20 # HP: 10, ダメージ量: -10 が入力されれば 20 が出力されるべき
    @test HP計算(0, 10) === -10  # HP: 0, ダメージ量: 10 が入力されれば -10 が出力されるべき
end

"""
メイン関数
"""
function main()
    モンスターHP = 30
    モンスター防御力 = 10
    勇者攻撃力 = 10

    println("モンスターに遭遇した！")
    println("戦闘開始！")

    for _ in 1:3
        println("----------")
        println("勇者の攻撃！")
        モンスターダメージ = ダメージ計算(勇者攻撃力, モンスター防御力)
        モンスターHP = HP計算(モンスターHP, モンスターダメージ)
        println("モンスターは $(モンスターダメージ) のダメージを受けた！")
        println("モンスターの残りHP：$(モンスターHP)")
    end

    println("戦闘に勝利した！")
end

main()

Test Summary: | Pass  Total
ダメージ計算  |    3      3
Test Summary: | Pass  Total
HP計算        |    3      3
モンスターに遭遇した！
戦闘開始！
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：20
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：10
----------
勇者の攻撃！
モンスターは 10 のダメージを受けた！
モンスターの残りHP：0
戦闘に勝利した！
